<a href="https://colab.research.google.com/github/saptarshidatta96/MTech_Sem3/blob/main/FCN_for_Semantic_Segmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [2]:
!unzip "/content/gdrive/MyDrive/cityscapes_data.zip" -d "/content/" 

Archive:  /content/gdrive/MyDrive/cityscapes_data.zip
   creating: /content/cityscapes_data/
   creating: /content/cityscapes_data/train/
  inflating: /content/cityscapes_data/train/1.jpg  
  inflating: /content/cityscapes_data/train/10.jpg  
  inflating: /content/cityscapes_data/train/100.jpg  
  inflating: /content/cityscapes_data/train/1000.jpg  
  inflating: /content/cityscapes_data/train/1001.jpg  
  inflating: /content/cityscapes_data/train/1002.jpg  
  inflating: /content/cityscapes_data/train/1003.jpg  
  inflating: /content/cityscapes_data/train/1004.jpg  
  inflating: /content/cityscapes_data/train/1005.jpg  
  inflating: /content/cityscapes_data/train/1006.jpg  
  inflating: /content/cityscapes_data/train/1007.jpg  
  inflating: /content/cityscapes_data/train/1008.jpg  
  inflating: /content/cityscapes_data/train/1009.jpg  
  inflating: /content/cityscapes_data/train/101.jpg  
  inflating: /content/cityscapes_data/train/1010.jpg  
  inflating: /content/cityscapes_data/train/

In [3]:
import cv2
from google.colab.patches import cv2_imshow
import numpy as np
import os
import shutil
from keras.models import Sequential,Model
from keras.layers import Convolution2D, ZeroPadding2D, MaxPooling2D, Conv2DTranspose, Cropping2D
from keras.layers import Input, Add, Dropout, Permute, add

In [4]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

Instructions for updating:
non-resource variables are not supported in the long term


In [18]:
if os.path.exists('/content/train') == False:
  os.mkdir('/content/train')
if os.path.exists('/content/val/') == False:
  os.mkdir('/content/val/')



In [19]:
def preprocess_data(path):
  
  for folder in os.listdir(path):
      img_path = os.path.join(path, folder)
      for img in os.listdir(img_path):
  
        img_src = os.path.join(img_path, img)
        imgg = cv2.imread(img_src)
        height, width, channels = imgg.shape
        x = imgg[0:height, 0:width//2]
        y = imgg[0:height, width//2:width]
        #cv2.imwrite('/content/ctscp_data/{}/label/{}'.format(folder, img), y)
        cv2.imwrite('/content/{}/{}'.format(folder, img),x)


In [20]:
preprocess_data('/content/cityscapes_data')

In [ ]:
#shutil.rmtree('/content/cityscapes_data')

In [21]:
'''
You should not edit helper.py as part of your submission.

This file is used primarily to download vgg if it has not yet been,
give you the progress of the download, get batches for your training,
as well as around generating and saving the image outputs.
'''

import re
import random
import numpy as np
import os.path
import scipy.misc
import shutil
import zipfile
import time
from glob import glob
from urllib.request import urlretrieve
from tqdm import tqdm


class DLProgress(tqdm):
	"""
	Report download progress to the terminal.
	:param tqdm: Information fed to the tqdm library to estimate progress.
	"""
	last_block = 0

	def hook(self, block_num=1, block_size=1, total_size=None):
		"""
		Store necessary information for tracking progress.
		:param block_num: current block of the download
		:param block_size: size of current block
		:param total_size: total download size, if known
		"""
		self.total = total_size
		self.update((block_num - self.last_block) * block_size)  # Updates progress
		self.last_block = block_num


def maybe_download_pretrained_vgg(data_dir):
	"""
	Download and extract pretrained vgg model if it doesn't exist
	:param data_dir: Directory to download the model to
	"""
	vgg_filename = 'vgg.zip'
	vgg_path = os.path.join(data_dir, 'vgg')
	vgg_files = [
		os.path.join(vgg_path, 'variables/variables.data-00000-of-00001'),
		os.path.join(vgg_path, 'variables/variables.index'),
		os.path.join(vgg_path, 'saved_model.pb')]

	missing_vgg_files = [vgg_file for vgg_file in vgg_files if not os.path.exists(vgg_file)]
	if missing_vgg_files:
		# Clean vgg dir
		if os.path.exists(vgg_path):
			shutil.rmtree(vgg_path)
		os.makedirs(vgg_path)

		# Download vgg
		print('Downloading pre-trained vgg model...')
		with DLProgress(unit='B', unit_scale=True, miniters=1) as pbar:
			urlretrieve(
				'https://s3-us-west-1.amazonaws.com/udacity-selfdrivingcar/vgg.zip',
				os.path.join(vgg_path, vgg_filename),
				pbar.hook)

		# Extract vgg
		print('Extracting model...')
		zip_ref = zipfile.ZipFile(os.path.join(vgg_path, vgg_filename), 'r')
		zip_ref.extractall(data_dir)
		zip_ref.close()

		# Remove zip file to save space
		os.remove(os.path.join(vgg_path, vgg_filename))


def gen_batch_function(data_folder, image_shape):
	"""
	Generate function to create batches of training data
	:param data_folder: Path to folder that contains all the datasets
	:param image_shape: Tuple - Shape of image
	:return:
	"""
	def get_batches_fn(batch_size):
		"""
		Create batches of training data
		:param batch_size: Batch Size
		:return: Batches of training data
		"""
		# Grab image and label paths
		image_paths = glob(os.path.join(data_folder, 'image_2', '*.png'))
		label_paths = {
			re.sub(r'_(lane|road)_', '_', os.path.basename(path)): path
			for path in glob(os.path.join(data_folder, 'gt_image_2', '*_road_*.png'))}
		background_color = np.array([255, 0, 0])

		# Shuffle training data
		random.shuffle(image_paths)
		# Loop through batches and grab images, yielding each batch
		for batch_i in range(0, len(image_paths), batch_size):
			images = []
			gt_images = []
			for image_file in image_paths[batch_i:batch_i+batch_size]:
				gt_image_file = label_paths[os.path.basename(image_file)]
				# Re-size to image_shape
				image = scipy.misc.imresize(scipy.misc.imread(image_file), image_shape)
				gt_image = scipy.misc.imresize(scipy.misc.imread(gt_image_file), image_shape)

				# Create "one-hot-like" labels by class
				gt_bg = np.all(gt_image == background_color, axis=2)
				gt_bg = gt_bg.reshape(*gt_bg.shape, 1)
				gt_image = np.concatenate((gt_bg, np.invert(gt_bg)), axis=2)

				images.append(image)
				gt_images.append(gt_image)

			yield np.array(images), np.array(gt_images)
	return get_batches_fn


def gen_test_output(sess, logits, keep_prob, image_pl, data_folder, image_shape):
	"""
	Generate test output using the test images
	:param sess: TF session
	:param logits: TF Tensor for the logits
	:param keep_prob: TF Placeholder for the dropout keep probability
	:param image_pl: TF Placeholder for the image placeholder
	:param data_folder: Path to the folder that contains the datasets
	:param image_shape: Tuple - Shape of image
	:return: Output for for each test image
	"""
	for image_file in glob(os.path.join(data_folder, 'image_2', '*.png')):
		image = scipy.misc.imresize(scipy.misc.imread(image_file), image_shape)

		# Run inference
		im_softmax = sess.run(
			[tf.nn.softmax(logits)],
			{keep_prob: 1.0, image_pl: [image]})
		# Splice out second column (road), reshape output back to image_shape
		im_softmax = im_softmax[0][:, 1].reshape(image_shape[0], image_shape[1])
		# If road softmax > 0.5, prediction is road
		segmentation = (im_softmax > 0.5).reshape(image_shape[0], image_shape[1], 1)
		# Create mask based on segmentation to apply to original image
		mask = np.dot(segmentation, np.array([[0, 255, 0, 127]]))
		mask = scipy.misc.toimage(mask, mode="RGBA")
		street_im = scipy.misc.toimage(image)
		street_im.paste(mask, box=None, mask=mask)

		yield os.path.basename(image_file), np.array(street_im)


def save_inference_samples(runs_dir, data_dir, sess, image_shape, logits, keep_prob, input_image):
	"""
	Save test images with semantic masks of lane predictions to runs_dir.
	:param runs_dir: Directory to save output images
	:param data_dir: Path to the directory that contains the datasets
	:param sess: TF session
	:param image_shape: Tuple - Shape of image
	:param logits: TF Tensor for the logits
	:param keep_prob: TF Placeholder for the dropout keep probability
	:param input_image: TF Placeholder for the image placeholder
	"""
	# Make folder for current run
	output_dir = os.path.join(runs_dir, str(time.time()))
	if os.path.exists(output_dir):
		shutil.rmtree(output_dir)
	os.makedirs(output_dir)

	# Run NN on test images and save them to HD
	print('Training Finished. Saving test images to: {}'.format(output_dir))
	image_outputs = gen_test_output(
		sess, logits, keep_prob, input_image, os.path.join(data_dir, 'data_road/testing'), image_shape)
	for name, image in image_outputs:
		scipy.misc.imsave(os.path.join(output_dir, name), image)

In [22]:
EPOCHS = 60
BATCH_SIZE = 25
NUM_CLASSES=29
IS_TRAIN = True
KEEP_PROB = 0.5
LEARNING_RATE = 0.0005
L2_REG = 1e-2
STDEV = 1e-3

DATA_DIR = '/content/train'
VIDEO_DIR = 'data\leftImg8bit_demoVideo\leftImg8bit\demoVideo\stuttgart_02'
RUNS_DIR = 'runs'

DEFAULT_SAVE_DIR = ''
PRETRAINED_MODEL_DIR = ''

In [23]:
import cv2
from collections import namedtuple
import random
import numpy as np
import os.path
import scipy.misc
import os
from glob import glob
from sklearn.utils import shuffle
import random
import time

def get_label_info():
    """
    Retrieve the class names and label values for the selected dataset.
    Must be in CSV format!
    # Arguments
        csv_path: The file path of the class dictionairy

    # Returns
        Two lists: one for the class names and the other for the label values
    """

    # a label and all meta information
    Label = namedtuple('Label', [

        'name',  # The identifier of this label, e.g. 'car', 'person', ... .
        # We use them to uniquely name a class

        'id',  # An integer ID that is associated with this label.
        # The IDs are used to represent the label in ground truth images
        # An ID of -1 means that this label does not have an ID and thus
        # is ignored when creating ground truth images (e.g. license plate).
        # Do not modify these IDs, since exactly these IDs are expected by the
        # evaluation server.

        'trainId',  # Feel free to modify these IDs as suitable for your method. Then create
        # ground truth images with train IDs, using the tools provided in the
        # 'preparation' folder. However, make sure to validate or submit results
        # to our evaluation server using the regular IDs above!
        # For trainIds, multiple labels might have the same ID. Then, these labels
        # are mapped to the same class in the ground truth images. For the inverse
        # mapping, we use the label that is defined first in the list below.
        # For example, mapping all void-type classes to the same ID in training,
        # might make sense for some approaches.
        # Max value is 255!

        'category',  # The name of the category that this label belongs to

        'categoryId',  # The ID of this category. Used to create ground truth images
        # on category level.

        'hasInstances',  # Whether this label distinguishes between single instances or not

        'ignoreInEval',  # Whether pixels having this class as ground truth label are ignored
        # during evaluations or not

        'color',  # The color of this label
    ])

    labels = [
        #       name                     id    trainId   category            catId     hasInstances   ignoreInEval   color
        Label('unlabeled', 0, 255, 'void', 0, False, True, (0, 0, 0)),
        Label('ego vehicle', 1, 255, 'void', 0, False, True, (0, 0, 0)),
        Label('rectification border', 2, 255, 'void', 0, False, True, (0, 0, 0)),
        Label('out of roi', 3, 255, 'void', 0, False, True, (0, 0, 0)),
        Label('static', 4, 255, 'void', 0, False, True, (0, 0, 0)),
        Label('dynamic', 5, 255, 'void', 0, False, True, (111, 74, 0)),
        Label('ground', 6, 255, 'void', 0, False, True, (81, 0, 81)),
        Label('road', 7, 0, 'flat', 1, False, False, (128, 64, 128)),
        Label('sidewalk', 8, 1, 'flat', 1, False, False, (244, 35, 232)),
        Label('parking', 9, 255, 'flat', 1, False, True, (250, 170, 160)),
        Label('rail track', 10, 255, 'flat', 1, False, True, (230, 150, 140)),
        Label('building', 11, 2, 'construction', 2, False, False, (70, 70, 70)),
        Label('wall', 12, 3, 'construction', 2, False, False, (102, 102, 156)),
        Label('fence', 13, 4, 'construction', 2, False, False, (190, 153, 153)),
        Label('guard rail', 14, 255, 'construction', 2, False, True, (180, 165, 180)),
        Label('bridge', 15, 255, 'construction', 2, False, True, (150, 100, 100)),
        Label('tunnel', 16, 255, 'construction', 2, False, True, (150, 120, 90)),
        Label('pole', 17, 5, 'object', 3, False, False, (153, 153, 153)),
        Label('polegroup', 18, 255, 'object', 3, False, True, (153, 153, 153)),
        Label('traffic light', 19, 6, 'object', 3, False, False, (250, 170, 30)),
        Label('traffic sign', 20, 7, 'object', 3, False, False, (220, 220, 0)),
        Label('vegetation', 21, 8, 'nature', 4, False, False, (107, 142, 35)),
        Label('terrain', 22, 9, 'nature', 4, False, False, (152, 251, 152)),
        Label('sky', 23, 10, 'sky', 5, False, False, (70, 130, 180)),
        Label('person', 24, 11, 'human', 6, True, False, (220, 20, 60)),
        Label('rider', 25, 12, 'human', 6, True, False, (255, 0, 0)),
        Label('car', 26, 13, 'vehicle', 7, True, False, (0, 0, 142)),
        Label('truck', 27, 14, 'vehicle', 7, True, False, (0, 0, 70)),
        Label('bus', 28, 15, 'vehicle', 7, True, False, (0, 60, 100)),
        Label('caravan', 29, 255, 'vehicle', 7, True, True, (0, 0, 90)),
        Label('trailer', 30, 255, 'vehicle', 7, True, True, (0, 0, 110)),
        Label('train', 31, 16, 'vehicle', 7, True, False, (0, 80, 100)),
        Label('motorcycle', 32, 17, 'vehicle', 7, True, False, (0, 0, 230)),
        Label('bicycle', 33, 18, 'vehicle', 7, True, False, (119, 11, 32)),
        Label('license plate', -1, -1, 'vehicle', 7, False, True, (0, 0, 142)),
    ]

    seen = set()
    label_list = list(map(lambda x : x[7], labels))
    label_values = [x for x in label_list if not (x in seen or seen.add(x))]

    return label_values

def get_data(data_path):
    train_path = data_path + '/leftImg8bit/train/'
    trainy_path = data_path + '/sky-data/train/'

    val_path = data_path + '/leftImg8bit/val/'
    valy_path = data_path + '/sky-data/val/'

    train_batch = glob(os.path.join(train_path, '*/*.png'))

    trainy_batch = glob(os.path.join(trainy_path, '*/*color.png'))
    val_batch = glob(os.path.join(val_path, '*/*.png'))

    valy_batch = glob(os.path.join(valy_path, '*/*color.png'))

    X_train = []
    y_train = []
    X_val = []
    y_val = []

    print('Loading X_Train..')
    for sample in train_batch:
        img_path = sample
        x = load_image(img_path)
        x = cv2.resize(x, dsize=(512, 256))
        X_train.append(x)

    print('Loading Y_Train..')
    for sample in trainy_batch:
        img_path = sample
        x = load_image(img_path)
        x = cv2.resize(x, dsize=(512, 256))
        y_train.append(x)

    print('Loading X_Validation..')
    for sample in val_batch:
        img_path = sample
        x = load_image(img_path)
        x = cv2.resize(x, dsize=(512, 256))
        X_val.append(x)

    print('Loading Y_Validation..')
    for sample in valy_batch:
        img_path = sample
        x = load_image(img_path)
        x = cv2.resize(x, dsize=(512, 256))
        y_val.append(x)

    return X_train, y_train, X_val, y_val

def one_hot_it(label, label_values):
    semantic_encoding = []
    # c = np.logical_and(np.not_equal(label, label_values[0]), np.not_equal(label, label_values[1]))
    # mask = np.any(c, axis=-1)
    # semantic_map.append(mask)
    for color in label_values:
        equality = np.equal(label, color)
        class_map = np.all(equality, axis=-1)
        semantic_encoding.append(class_map)
    semantic_map = np.stack(semantic_encoding, axis=-1)

    return semantic_map


def reverse_one_hot(image):
    x = np.argmax(image, axis=-1)
    return x


def colour_code_segmentation(image, label_values):

    label_matrix = np.array(label_values)
    x = label_matrix[image.astype(int)]

    return x


def load_image(path):
    image = cv2.cvtColor(cv2.imread(path, -1), cv2.COLOR_BGR2RGB)
    return image


def load_annotation(path):
    image = cv2.imread(path, -1)
    return image


def flip_image(image, measurement, flip_probability=1.0):
    if random.random() <= flip_probability:
        image = cv2.flip(image, 1)
        measurement *= -1
    return image, measurement


# Randomly crop the image to a specific size. For data augmentation
def random_crop(image, label, crop_height, crop_width):
    if (image.shape[0] != label.shape[0]) or (image.shape[1] != label.shape[1]):
        raise Exception('Image and label must have the same dimensions!')

    if (crop_width <= image.shape[1]) and (crop_height <= image.shape[0]):
        x = random.randint(0, image.shape[1] - crop_width)
        y = random.randint(0, image.shape[0] - crop_height)

        if len(label.shape) == 3:
            return image[y:y + crop_height, x:x + crop_width, :], label[y:y + crop_height, x:x + crop_width, :]
        else:
            return image[y:y + crop_height, x:x + crop_width, :], label[y:y + crop_height, x:x + crop_width]
    else:
        raise Exception('Crop shape (%d, %d) exceeds image dimensions (%d, %d)!' % (
            crop_height, crop_width, image.shape[0], image.shape[1]))


def random_shadow(image):
    """
    Generates and adds random shadow
    """
    rand_width_scal_1 = np.random.rand()
    IMAGE_HEIGHT, IMAGE_WIDTH, _ = image.shape
    x1, y1 = IMAGE_WIDTH * rand_width_scal_1, 0
    rand_width_scal_2 = np.random.rand()
    x2, y2 = IMAGE_WIDTH * rand_width_scal_2, IMAGE_HEIGHT
    xn, yn = np.mgrid[0:IMAGE_HEIGHT, 0:IMAGE_WIDTH]
    mask = np.zeros_like(image[:, :, 1])
    mask[(yn - y1) * (x2 - x1) - (y2 - y1) * (xn - x1) > 0] = 1

    cond = mask == np.random.randint(2)
    s_ratio = np.random.uniform(low=0.2, high=0.5)

    hls = cv2.cvtColor(image, cv2.COLOR_RGB2HLS)
    hls[:, :, 1][cond] = hls[:, :, 1][cond] * s_ratio
    return cv2.cvtColor(hls, cv2.COLOR_HLS2RGB)

def random_brightness(image):
    """
    Randomly adjust brightness of the image.
    HSV (Hue, Saturation, Value) is also called HSB ('B' for Brightness).
    """

    hsv_channel = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)
    brightness_scalar = np.random.rand()
    ratio = 1.0 + 0.4 * (brightness_scalar - 0.5)
    hsv_channel[:, :, 2] = hsv_channel[:, :, 2] * ratio
    return cv2.cvtColor(hsv_channel, cv2.COLOR_HSV2RGB)


def rotation(input_image, output_image, degrees):
    angle = random.uniform(-1*degrees, degrees)
    M = cv2.getRotationMatrix2D((input_image.shape[1]//2, input_image.shape[0]//2), angle, 1.0)
    input_image = cv2.warpAffine(input_image, M, (input_image.shape[1], input_image.shape[0]), flags=cv2.INTER_NEAREST)
    output_image = cv2.warpAffine(output_image, M, (output_image.shape[1], output_image.shape[0]), flags=cv2.INTER_NEAREST)
    return input_image, output_image

def brightness(image, bright_0_1):
            factor = 1.0 + random.uniform(-1.0*bright_0_1, bright_0_1)
            table = np.array([((i / 255.0) * factor) * 255 for i in np.arange(0, 256)]).astype(np.uint8)
            image = cv2.LUT(image, table)
            return image


def data_augmentation(input_image, output_image):

    if random.randint(0, 1):
        input_image = brightness(input_image, 0.5)

    if random.randint(0, 1):
        input_image = random_shadow(input_image)


    return input_image, output_image


def gen_batch_function(samplesX, samplesY, label_values, batch_size=1, is_train=True):
    """
    F function to create batches of training data
    :param data_folder: Path to folder that contains all the datasets
    :param image_shape: Tuple - Shape of image
    :return:"""

    samplesX, samplesY = shuffle(samplesX, samplesY)

    # Shuffle training data

    num_samples = len(samplesX)

    if (batch_size == -1):
        batch_size = num_samples

    # Loop through batches and grab images, yielding each batch
    for batch_i in range(0, num_samples, batch_size):

        X_train = samplesX[batch_i:batch_i + batch_size]
        y_train = samplesY[batch_i:batch_i + batch_size]

        # preprocessing if required
        X_f = []
        y_f = []
        for x, y in zip(X_train, y_train):
            if is_train:
                x, y = data_augmentation(x, y)

            y = np.float32(one_hot_it(y, label_values=label_values))
            X_f.append(x)
            y_f.append(y)

        X_f = np.float32(X_f)
        y_f = np.float32(y_f)
        yield X_f, y_f

def pipeline_final(img, sess, logits, keep_prob, input_image, image_shape, label_values, num_classes=29):
    channel = 1
    img= cv2.resize(img, dsize=image_shape)
    img = np.array([img])
    softmax_ = loss = sess.run([logits],
                       feed_dict={input_image: img, keep_prob:1})
    logits_ = (softmax_[0].reshape(1,image_shape[1],image_shape[0],num_classes))
    output_image = reverse_one_hot(logits_[0])

    out_vis_image = colour_code_segmentation(output_image, label_values)

    a = cv2.cvtColor(np.uint8(out_vis_image), channel)

    b = cv2.cvtColor(np.uint8(img[0]), channel)

    added_image = cv2.addWeighted(a, 1, b, 1, channel)
    added_image = cv2.resize(added_image, image_shape)

    return added_image


def process(media_dir, sess, logits, keep_prob, input_image, image_shape, label_values):
    img = load_image(media_dir)
    img = pipeline_final(img, sess, logits, keep_prob, input_image, image_shape, label_values)
    return img


def gen_test_output(sess, logits, keep_prob, input_image, data_folder, image_shape, label_values):

    for image_file in glob(os.path.join(data_folder, '*.png')):
        image = process(image_file, sess, logits, keep_prob, input_image, image_shape, label_values)
        yield os.path.basename(image_file), image


def save_inference_samples(runs_dir, data_dir, sess, image_shape, logits, keep_prob, input_image, label_values):
	"""
	Save test images with semantic masks of lane predictions to runs_dir.
	:param runs_dir: Directory to save output images
	:param data_dir: Path to the directory that contains the datasets
	:param sess: TF session
	:param image_shape: Tuple - Shape of image
	:param logits: TF Tensor for the logits
	:param keep_prob: TF Placeholder for the dropout keep probability
	:param input_image: TF Placeholder for the image placeholder
	"""
	# Make folder for current run
	output_dir = os.path.join(runs_dir, str(time.time()))
	if os.path.exists(output_dir):
		shutil.rmtree(output_dir)
	os.makedirs(output_dir)

	# Run NN on test images and save them to HD
	print('Saving test images to: {}'.format(output_dir))
	image_outputs = gen_test_output(

		sess, logits, keep_prob, input_image, data_dir, image_shape, label_values)
	for name, image in image_outputs:
		scipy.misc.imsave(os.path.join(output_dir, name), image)

In [24]:
#!/usr/bin/env python3
import os.path
import helper
import warnings
from distutils.version import LooseVersion



# Check TensorFlow Version
assert LooseVersion(tf.__version__) >= LooseVersion('1.0'), 'Please use TensorFlow version 1.0 or newer.  You are using {}'.format(tf.__version__)
print('TensorFlow Version: {}'.format(tf.__version__))

# Check for a GPU
if not tf.test.gpu_device_name():
    warnings.warn('No GPU found. Please use a GPU to train your neural network.')
else:
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))


def load_vgg(sess, vgg_path):
    """
    Load Pretrained VGG Model into TensorFlow.
    :param sess: TensorFlow Session
    :param vgg_path: Path to vgg folder, containing "variables/" and "saved_model.pb"
    :return: Tuple of Tensors from VGG model (image_input, keep_prob, layer3_out, layer4_out, layer7_out)
    """
    # TODO: Implement function
    #   Use tf.saved_model.loader.load to load the model and weights
    vgg_tag = 'vgg16'
    vgg_input_tensor_name = 'image_input:0'
    vgg_keep_prob_tensor_name = 'keep_prob:0'
    vgg_layer3_out_tensor_name = 'layer3_out:0'
    vgg_layer4_out_tensor_name = 'layer4_out:0'
    vgg_layer7_out_tensor_name = 'layer7_out:0'

    graph = tf.get_default_graph()
    tf.saved_model.loader.load(sess, [vgg_tag], vgg_path)
    input = graph.get_tensor_by_name(vgg_input_tensor_name)
    keep_prob = graph.get_tensor_by_name(vgg_keep_prob_tensor_name)
    layer3 = graph.get_tensor_by_name(vgg_layer3_out_tensor_name)
    layer4 = graph.get_tensor_by_name(vgg_layer4_out_tensor_name)
    layer7 = graph.get_tensor_by_name(vgg_layer7_out_tensor_name)
    return input, keep_prob, layer3, layer4, layer7





def layers(vgg_layer3_out, vgg_layer4_out, vgg_layer7_out, num_classes):
    """
    Create the layers for a fully convolutional network.  Build skip-layers using the vgg layers.
    :param vgg_layer3_out: TF Tensor for VGG Layer 3 output
    :param vgg_layer4_out: TF Tensor for VGG Layer 4 output
    :param vgg_layer7_out: TF Tensor for VGG Layer 7 output
    :param num_classes: Number of classes to classify
    :return: The Tensor for the last layer of output
    """
    # TODO: Implement function
    l7_conv = tf.layers.conv2d(vgg_layer7_out, num_classes, 1, 1,
                                       padding='same', kernel_initializer= tf.random_normal_initializer(stddev=STDEV),
                                       name='conv_1_1_1',activation = tf.nn.relu)


    conv1 = tf.layers.conv2d_transpose(l7_conv, num_classes, 4, 2,
                                        padding='same', kernel_initializer= tf.random_normal_initializer(stddev=STDEV),
                                        name='conv_1_1_2',activation = tf.nn.relu)

    l4_conv = tf.layers.conv2d(vgg_layer4_out, num_classes, 1, 1,
                                       padding='same', kernel_initializer= tf.random_normal_initializer(stddev=STDEV),
                                       name='conv_1_1_3',activation = tf.nn.relu)

    skip_1 = tf.add(conv1, l4_conv, name='conv_1_1_4')
    #output = tf.layers.batch_normalization(output)
    #output = keras.layers.UpSampling2D(size=(2,2),data_format=None,interpolation='bilinear')(output)

    conv2 = tf.layers.conv2d_transpose(skip_1, num_classes, 4, 2,
                                       padding='same', kernel_initializer= tf.random_normal_initializer(stddev=STDEV),
                                       name='conv_1_1_5',activation = tf.nn.relu)
    l3_conv = tf.layers.conv2d(vgg_layer3_out, num_classes, 1, 1,
                                       padding='same', kernel_initializer= tf.random_normal_initializer(stddev=STDEV),
                                       name='conv_1_1_6',activation = tf.nn.relu)
    skip_3 = tf.add(conv2, l3_conv,  name='conv_1_1_7')

    output = tf.layers.conv2d_transpose(skip_3, num_classes, 16, 8,
                                        padding='same', kernel_initializer= tf.random_normal_initializer(stddev=STDEV),
                                        name='conv_1_1_8',activation = tf.nn.relu)

    return output




def optimize(nn_last_layer, correct_label, learning_rate, num_classes):
    """
    Build the TensorFLow loss and optimizer operations.
    :param nn_last_layer: TF Tensor of the last layer in the neural network
    :param correct_label: TF Placeholder for the correct label image
    :param learning_rate: TF Placeholder for the learning rate
    :param num_classes: Number of classes to classify
    :return: Tuple of (logits, train_op, cross_entropy_loss)
    """
    # TODO: Implement function

    logits = tf.reshape(nn_last_layer, (-1, num_classes), name='logits')
    labels = tf.reshape(correct_label, (-1, num_classes), name='labels')
    cross_entropy_loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=labels))

    train_op = tf.train.AdamOptimizer(learning_rate=0.0005).minimize(cross_entropy_loss)

    return logits, train_op, cross_entropy_loss


import time


def train_nn(sess, epochs, batch_size, get_batches_fn, train_op, cross_entropy_loss, input_image,
                 correct_label, keep_prob, learning_rate, X_train, y_train, label_values, X_val, y_val):
    """
    Train neural network and print out the loss during training.
    :param sess: TF Session
    :param epochs: Number of epochs
    :param batch_size: Batch size
    :param get_batches_fn: Function to get batches of training data.  Call using get_batches_fn(batch_size)
    :param train_op: TF Operation to train the neural network
    :param cross_entropy_loss: TF Tensor for the amount of loss
    :param input_image: TF Placeholder for input images
    :param correct_label: TF Placeholder for label images
    :param keep_prob: TF Placeholder for dropout keep probability
    :param learning_rate: TF Placeholder for learning rate
    """

    for epoch in range(epochs):
        print('epoch : ', epoch)
        for image, targets in get_batches_fn(X_train, y_train, label_values, batch_size):
            _, loss = sess.run([train_op, cross_entropy_loss],
                               feed_dict={input_image: image, correct_label: targets, keep_prob: KEEP_PROB,
                                          learning_rate: LEARNING_RATE})
        # Print data on the learning process

            print("Epoch: {}".format(epoch + 1), "/ {}".format(epochs), " Loss: {:.3f}".format(loss))

        mean_loss = []
        for image, targets in get_batches_fn(X_val, y_val, label_values, batch_size, is_train=False):
            loss = sess.run([cross_entropy_loss],
                            feed_dict={input_image: image, correct_label: targets, keep_prob: 1})
            mean_loss.append(loss)

        mean_loss_ = np.mean(np.array(mean_loss))
        print("Epoch: {}".format(epoch + 1), "/ {}".format(epochs), " Validation Loss: {:.3f}".format(mean_loss_))
        saver = tf.train.Saver()
        print('saving model')
        saver.save(sess, './model')


#tests.test_train_nn(train_nn)


def run():
    num_classes = NUM_CLASSES
    image_shape = (512, 256)  # Cityscapes dataset should be scaled (Using GTX-1080TI)
    data_dir = DATA_DIR
    video_dir = VIDEO_DIR
    runs_dir = RUNS_DIR

    epochs = EPOCHS
    batch_size = BATCH_SIZE
    is_train = IS_TRAIN
    # Download pretrained vgg model
    maybe_download_pretrained_vgg(data_dir)
    label_values = get_label_info()

    # You'll need a GPU with at least 10 teraFLOPS to train on.
    #  https://www.cityscapes-dataset.com/

    print('run')
    tf.reset_default_graph()
    with tf.Session() as sess:
        # Path to vgg model
        vgg_path = os.path.join(data_dir, 'vgg')
        # Create function to get batches
        get_batches_fn = gen_batch_function

        input, keep_prob, layer3, layer4, layer7 = load_vgg(sess, vgg_path)
        print(input)
        output = layers(layer3, layer4, layer7, num_classes)
        correct_label = tf.placeholder(dtype=tf.float32, shape=(None, None, None, num_classes))
        learning_rate = tf.placeholder(dtype=tf.float32)
        logits, train_op, cross_entropy_loss = optimize(output, correct_label, learning_rate, num_classes)
        logits = tf.nn.softmax(logits, name='softmax')
        sess.run(tf.global_variables_initializer())
        saver = tf.train.Saver()  # Simple model saver

        if not is_train:
            saver.restore(sess, tf.train.latest_checkpoint('.'))
            save_inference_samples(runs_dir, video_dir, sess, image_shape, logits, keep_prob, input, label_values)
        else:
            X_train, y_train, X_val, y_val = get_data(data_dir)
            train_nn(sess, epochs, batch_size, get_batches_fn, train_op, cross_entropy_loss, input, correct_label,
                         keep_prob, learning_rate, X_train, y_train, label_values, X_val, y_val)
            save_inference_samples(runs_dir, video_dir, sess, image_shape, logits, keep_prob, input,
                                                     label_values)


if __name__ == '__main__':
    run()

TensorFlow Version: 2.8.0
Default GPU Device: /device:GPU:0


997MB [01:07, 14.8MB/s]                           


Extracting model...
run
Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.loader.load or tf.compat.v1.saved_model.load. There will be a new function for importing SavedModels in Tensorflow 2.0.
INFO:tensorflow:Restoring parameters from /content/train/vgg/variables/variables
Tensor("image_input:0", shape=(?, ?, ?, 3), dtype=float32)
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:61: UserWarning: `tf.layers.conv2d` is deprecated and will be removed in a future version. Please Use `tf.keras.layers.Conv2D` instead.
/usr/local/lib/python3.7/dist-packages/keras/legacy_tf_layers/convolutional.py:575: UserWarning: `layer.apply` is deprecated and will be removed in a future version. Please use `layer.__call__` method instead.
  return layer.apply(inputs)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:66: UserWarning: `tf.layers.conv2d_transpose` is deprecated and will be removed in a future version. Please Use `tf.keras.layers.Conv2DTranspose` instead.
/usr/local/lib/python3.7/dist-packages/keras/legacy_tf_layers/convolutional.py:1736: UserWarning: `layer.apply` is deprecated and will be removed in a future version. Please use `layer.__call__` method instead.
  return layer.apply(inputs)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:70: UserWarning: `tf.layers.conv2d` is deprecat

Loading X_Train..
Loading Y_Train..
Loading X_Validation..
Loading Y_Validation..
epoch :  0
Epoch: 1 / 60  Validation Loss: nan
saving model


/usr/local/lib/python3.7/dist-packages/numpy/core/fromnumeric.py:3441: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


epoch :  1
Epoch: 2 / 60  Validation Loss: nan
saving model
epoch :  2
Epoch: 3 / 60  Validation Loss: nan
saving model
epoch :  3
Epoch: 4 / 60  Validation Loss: nan
saving model
epoch :  4
Epoch: 5 / 60  Validation Loss: nan
saving model
epoch :  5
Epoch: 6 / 60  Validation Loss: nan
saving model
epoch :  6
Epoch: 7 / 60  Validation Loss: nan
saving model
epoch :  7
Epoch: 8 / 60  Validation Loss: nan
saving model
epoch :  8
Epoch: 9 / 60  Validation Loss: nan
saving model
epoch :  9
Epoch: 10 / 60  Validation Loss: nan
saving model
epoch :  10
Epoch: 11 / 60  Validation Loss: nan
saving model
epoch :  11
Epoch: 12 / 60  Validation Loss: nan
saving model
epoch :  12
Epoch: 13 / 60  Validation Loss: nan
saving model
epoch :  13
Epoch: 14 / 60  Validation Loss: nan
saving model
epoch :  14
Epoch: 15 / 60  Validation Loss: nan
saving model
epoch :  15
Epoch: 16 / 60  Validation Loss: nan
saving model
epoch :  16
Epoch: 17 / 60  Validation Loss: nan
saving model
epoch :  17
Epoch: 18 / 6

In [31]:
#!/usr/bin/env python3
import os.path
import helper
import warnings
from distutils.version import LooseVersion
import numpy as np
import argparse as parser

from moviepy.editor import VideoFileClip
import cv2

def load_image(path):
    image = cv2.cvtColor(cv2.imread(path, -1), cv2.COLOR_BGR2RGB)
    return image


def layers(vgg_layer3_out, vgg_layer4_out, vgg_layer7_out, num_classes):
    """
    Create the layers for a fully convolutional network.  Build skip-layers using the vgg layers.
    :param vgg_layer3_out: TF Tensor for VGG Layer 3 output
    :param vgg_layer4_out: TF Tensor for VGG Layer 4 output
    :param vgg_layer7_out: TF Tensor for VGG Layer 7 output
    :param num_classes: Number of classes to classify
    :return: The Tensor for the last layer of output
    """
    # TODO: Implement function
    l7_conv = tf.layers.conv2d(vgg_layer7_out, num_classes, 1, 1,
                                       padding='same', kernel_initializer= tf.random_normal_initializer(stddev=STDEV),
                                       name='conv_1_1_1',activation = tf.nn.relu)


    conv1 = tf.layers.conv2d_transpose(l7_conv, num_classes, 4, 2,
                                        padding='same', kernel_initializer= tf.random_normal_initializer(stddev=STDEV),
                                        name='conv_1_1_2',activation = tf.nn.relu)

    l4_conv = tf.layers.conv2d(vgg_layer4_out, num_classes, 1, 1,
                                       padding='same', kernel_initializer= tf.random_normal_initializer(stddev=STDEV),
                                       name='conv_1_1_3',activation = tf.nn.relu)

    skip_1 = tf.add(conv1, l4_conv, name='conv_1_1_4')
    #output = tf.layers.batch_normalization(output)
    #output = keras.layers.UpSampling2D(size=(2,2),data_format=None,interpolation='bilinear')(output)

    conv2 = tf.layers.conv2d_transpose(skip_1, num_classes, 4, 2,
                                       padding='same', kernel_initializer= tf.random_normal_initializer(stddev=STDEV),
                                       name='conv_1_1_5',activation = tf.nn.relu)
    l3_conv = tf.layers.conv2d(vgg_layer3_out, num_classes, 1, 1,
                                       padding='same', kernel_initializer= tf.random_normal_initializer(stddev=STDEV),
                                       name='conv_1_1_6',activation = tf.nn.relu)
    skip_3 = tf.add(conv2, l3_conv,  name='conv_1_1_7')

    output = tf.layers.conv2d_transpose(skip_3, num_classes, 16, 8,
                                        padding='same', kernel_initializer= tf.random_normal_initializer(stddev=STDEV),
                                        name='conv_1_1_8',activation = tf.nn.relu)

    return output

def load_vgg(sess, vgg_path):
    """
    Load Pretrained VGG Model into TensorFlow.
    :param sess: TensorFlow Session
    :param vgg_path: Path to vgg folder, containing "variables/" and "saved_model.pb"
    :return: Tuple of Tensors from VGG model (image_input, keep_prob, layer3_out, layer4_out, layer7_out)
    """
    # TODO: Implement function
    #   Use tf.saved_model.loader.load to load the model and weights
    vgg_tag = 'vgg16'
    vgg_input_tensor_name = 'image_input:0'
    vgg_keep_prob_tensor_name = 'keep_prob:0'
    vgg_layer3_out_tensor_name = 'layer3_out:0'
    vgg_layer4_out_tensor_name = 'layer4_out:0'
    vgg_layer7_out_tensor_name = 'layer7_out:0'

    graph = tf.get_default_graph()
    tf.saved_model.loader.load(sess, [vgg_tag], vgg_path)
    input = graph.get_tensor_by_name(vgg_input_tensor_name)
    keep_prob = graph.get_tensor_by_name(vgg_keep_prob_tensor_name)
    layer3 = graph.get_tensor_by_name(vgg_layer3_out_tensor_name)
    layer4 = graph.get_tensor_by_name(vgg_layer4_out_tensor_name)
    layer7 = graph.get_tensor_by_name(vgg_layer7_out_tensor_name)
    return input, keep_prob, layer3, layer4, layer7


def pipeline_final(img, is_video):
    channel = 1 if is_video else 4

    img = cv2.resize(img, dsize=(512, 256))
    img = np.array([img])
    softmax_   = sess.run([softmax],
                               feed_dict={input: img, keep_prob: 1})
    logits_ = (softmax_[0].reshape(1, 256, 512, 29))
    output_image = reverse_one_hot(logits_[0])

    print(output_image.shape)

    out_vis_image = colour_code_segmentation(output_image, label_values)

    a = cv2.cvtColor(np.uint8(out_vis_image), channel)

    b = cv2.cvtColor(np.uint8(img[0]), channel)

    added_image = cv2.addWeighted(a, 1, b, 1, channel)
    added_image = cv2.resize(added_image, dsize=(512, 256))

    return added_image

def pipeline_video(img):
    return pipeline_final(img, True)

def pipeline_img(img):
    return pipeline_final(img, False)

def process(media_dir, save_dir):
    global sess, softmax, label_values, input, keep_prob

    data_dir = '/content/train'
    num_classes = 29

    label_values = get_label_info()

    tf.reset_default_graph()
    sess = tf.Session()

    # Path to vgg model
    vgg_path = os.path.join(data_dir, 'vgg')
    # Create function to get batches

    input, keep_prob, layer3, layer4, layer7 = load_vgg(sess, vgg_path)
    output = layers(layer3, layer4, layer7, num_classes)

    sess.run(tf.global_variables_initializer())

    saver = tf.train.Saver()  # Simple model saver

    saver.restore(sess, tf.train.latest_checkpoint('.'))

    logits = tf.reshape(output, (-1, num_classes))
    softmax = tf.nn.softmax(logits, name='softmax')


    try:
        img = load_image(media_dir)
        output = os.path.join(save_dir, 'output_image.png')
        img = pipeline_img(img)
        cv2.imwrite(output, img)
    except Exception as ex:
        output = os.path.join(save_dir, 'output_video.mp4')
        clip1 = VideoFileClip(media_dir)
        white_clip = clip1.fl_image(pipeline_video)
        white_clip.write_videofile(output, audio=False)

if __name__ == '__main__':



        media_dir = '/content/train/10.jpg'
        save_dir = '/content/saved_op'
        

        crawler = process(media_dir= media_dir, save_dir= save_dir )

INFO:tensorflow:Restoring parameters from /content/train/vgg/variables/variables
INFO:tensorflow:Restoring parameters from ./model
(256, 512)


In [26]:
os.mkdir('/content/saved_op')